<h1><center>Hito 7</center></h1>

<h1><center>Impacto del Bit-Rate</center></h1>

&nbsp;

<center>
Álvaro Garcı́a Garcı́a<br>
Álvaro José Martı́nez Sánchez<br>
José Francisco Castillo Berenguel<br>
</center>
    
    
<hr style="border:1px solid gray"> </hr>


# Introducción


En el hito 6 se estudiaron los efectos de la latencia y su variación (jitter). No obstante, existen otros factores que también afectan a la conexión. En este hito se realizará el estudio de la tasa de bits (Bit rate). 

Este valor de bit-rate depende del ancho de banda y de la congestión de la red. La congestión de la red resulta ser un factor arbitrario en la práctica, pero el ancho de banda puede considerarse constante.

La idea consistirá en obtener datos sobre el efecto del bit rate para posteriormente realizar la emulación con e comando 
`tc`

# Estimación del bit-rate en un enlace de internet


## Fundamentos

## Consideraciones

Se pide realizar distintos tipos de ping donde se incluye realizarlos con el mayor y menor payload. Para que los ping se realicen satisfactoriamente es necesario tener en cuenta estos efectos

1. La cabecera de ICMPv4 ocupa 8 B.

2. El valor mínimo de payload es 0. Sin embargo no se contabilizan los tiampos para payload menores de 16B (24B con la cabecera).

3. El valor máximo de payload es 65507.


## Procedimiento

Vamos a realizar el mismo procedimiento para 3 conexiones distintas:

1. Localhost
2. Red local
3. Conexión internet

Para ello realizamos ping con el menor y mayor payload posible 16B y 65507B respectivamente. Una vez hecho, cargamos los resultados. Sin embargo al final del ping se ofrecen las estadísticas del mismo y por tanto no hay que realizar cálculos. Obtenemos los datos:

In [1]:
!export LC_NUMERIC=en_US.UTF-8

!tail -n 1 ping-minimo-localhost.dat | cut -f 2 -d "=" | cut -f 2 -d "/" > average-localhost.dat
!tail -n 1 ping-maximo-localhost.dat | cut -f 2 -d "=" | cut -f 2 -d "/" >> average-localhost.dat

!tail -n 1 ping-minimo-redlocal.dat | cut -f 2 -d "=" | cut -f 2 -d "/" > average-redlocal.dat
!tail -n 1 ping-maximo-redlocal.dat | cut -f 2 -d "=" | cut -f 2 -d "/" >> average-redlocal.dat

!tail -n 1 ping-minimo-internet.dat | cut -f 2 -d "=" | cut -f 2 -d "/" > average-internet.dat
!tail -n 1 ping-maximo-internet.dat | cut -f 2 -d "=" | cut -f 2 -d "/" >> average-internet.dat

!tail -n 1 ping-minimo-internet2.dat | cut -f 2 -d "=" | cut -f 2 -d "/" > average-internet2.dat
!tail -n 1 ping-maximo-internet2.dat | cut -f 2 -d "=" | cut -f 2 -d "/" >> average-internet2.dat

Cargamos los datos en python

In [15]:
import numpy as np
import matplotlib.pyplot as plt
    
rtt_localhost = np.loadtxt("average-localhost.dat")
rtt_redlocal = np.loadtxt("average-redlocal.dat")
rtt_internet = np.loadtxt("average-internet.dat")
rtt_internet2 = np.loadtxt("average-internet2.dat")

print("Tiempos RTT")
print("\nTiempo promedio localhost con payload de 16 B: ",rtt_localhost[0])
print("Tiempo promedio localhost con payload de 65507 B: ",rtt_localhost[1])

print("\nTiempo promedio al gateway con payload de 16 B: ",rtt_redlocal[0])
print("Tiempo promedio al gateway con payload de 65507 B: ",rtt_redlocal[1])

print("\nTiempo promedio en internet (1) con payload de 16 B: ",rtt_internet[0])
print("Tiempo promedio en internet (1) con payload de 65507 B: ",rtt_internet[1])

print("\nTiempo promedio en internet (2) con payload de 16 B: ",rtt_internet2[0])
print("Tiempo promedio en internet (2) con payload de 65507 B: ",rtt_internet2[1])


tp_localhost = rtt_localhost[0]/2
tp_redlocal = rtt_redlocal[0]/2
tp_internet = rtt_internet[0]/2
tp_internet2 = rtt_internet2[0]/2

print("Tiempos de propagación\n")

print("Tiempo de propagación en localhost: \t", tp_localhost)
print("Tiempo de propagación con el gateway: \t", tp_redlocal)
print("Tiempo de propagación en internet (1): \t", tp_internet)
print("Tiempo de propagación en internet (2): \t", tp_internet2)

transmision_time = lambda rtt, tp: (rtt - 2.0 * tp) / 2.0

tt_localhost = (rtt_localhost[1] - 2.0 * tp_localhost) / 2.0
tt_redlocal = (rtt_redlocal[1] - 2.0 * tp_redlocal) / 2.0
tt_internet = (rtt_internet[1] - 2.0 * tp_internet) / 2.0
tt_internet2 = (rtt_internet2[1] - 2.0 * tp_internet2) / 2.0

print("Tiempos de transmisión\n")
print("Tiempo de transmisión en localhost: \t", tt_localhost)
print("Tiempo de transmisión en redlocal: \t", tt_redlocal)
print("Tiempo de transmisión en internet (1): \t", tt_internet)
print("Tiempo de transmisión en internet (2): \t", tt_internet2)
print()

bitrate = lambda B, tt: (B/tt) 

#space = np.linspace(0,65507, 65507)
#plt.plot(space, bitrate(space, tt_localhost), 'g',linewidth=3, label='Localhost') 
#plt.plot(space, bitrate(space, tt_redlocal), 'b',linewidth=3, label='Red Local') 
#plt.plot(space, bitrate(space, tt_internet), 'r',linewidth=3, label='Internet')
#plt.plot(space, bitrate(space, tt_internet2), 'y',linewidth=3, label='Internet2')
#plt.ylabel(ylabel="Bit rate")
#plt.xlabel(xlabel="Número de bits enviados")
#plt.legend()
#plt.show()
#print(len(space))
#print(bitrate(space,10))

Tiempos RTT

Tiempo promedio localhost con payload de 16 B:  0.037
Tiempo promedio localhost con payload de 65507 B:  0.163

Tiempo promedio al gateway con payload de 16 B:  18.557
Tiempo promedio al gateway con payload de 65507 B:  45.203

Tiempo promedio en internet (1) con payload de 16 B:  27.731
Tiempo promedio en internet (1) con payload de 65507 B:  56.002

Tiempo promedio en internet (2) con payload de 16 B:  41.056
Tiempo promedio en internet (2) con payload de 65507 B:  104.125
Tiempos de propagación

Tiempo de propagación en localhost: 	 0.0185
Tiempo de propagación con el gateway: 	 9.2785
Tiempo de propagación en internet (1): 	 13.8655
Tiempo de propagación en internet (2): 	 20.528
Tiempos de transmisión
l
Tiempo de propagación en localhost: 	 0.063
Tiempo de propagación en redlocal: 	 13.323000000000002
Tiempo de propagación en internet (1): 	 14.1355
Tiempo de propagación en internet (2): 	 31.5345



Dado que localhost tiene un bitrate "elevado", se procede a realizar al comparativa descartando localhost

In [21]:

#plt.plot(space, bitrate(space, tt_redlocal), 'b',linewidth=3, label='Red Local') 
#plt.plot(space, bitrate(space, tt_internet), 'r',linewidth=3, label='Internet')
#plt.plot(space, bitrate(space, tt_internet2), 'y',linewidth=3, label='Internet2')
#plt.ylabel(ylabel="Bit rate")
#plt.xlabel(xlabel="Número de bits enviados")
#plt.legend()
#plt.show()

#print(bitrate(space, tt_internet))

# Simulación del enlace

Una vez se tienen los valores de bitrate, se procede a mejorar la simulación realizada en el hito anterior. Para ello se hace uso del comando  `tc`. Primero se calculan los bitrate. En este caso, se deben realizar con el valor B del maximo payload, puesto que los resultados de ping están condicionados a ese valor.

In [20]:
bitrate_localhost = bitrate(65507*8, tt_localhost)
bitrate_redlocal = bitrate(65507*8, tt_redlocal)
bitrate_internet = bitrate(65507*8, tt_internet)
bitrate_internet2 = bitrate(65507*8, tt_internet2)

print("Bitrate en localhost: ", bitrate_localhost)
print("Bitrate al gateway: ", bitrate_redlocal)
print("Bitrate en internet (1): ", bitrate_internet)
print("Bitrate en internet (2): ", bitrate_internet2)

Bitrate en localhost:  8318349.206349206
Bitrate al gateway:  39334.6843803948
Bitrate en internet (1):  37073.750486364115
Bitrate en internet (2):  16618.497201477745


Se procede a emular la latencia y la pérdida de paquetes. Para ello, primero se debe ver la configuración

In [6]:
!tc qdisc show dev lo 

qdisc noqueue 0: root refcnt 2 


Imponemos la regla para emular el bitrate

In [7]:
from getpass import getpass
!echo {getpass()} | sudo -S tc qdisc add dev lo root handle 1: tbf rate 4634kbit burst 32kbit limit 32kbit

········
[sudo] password for cobeguel: 

Se activa la regla del hito anterior

In [ ]:
!echo {getpass()} | sudo -S tc qdisc add dev lo parent 1:1 handle 10: netem delay 43.75ms 6.88ms 0.375% distribution normal

Visualizamos la configuración

In [ ]:
!echo {getpass()} | sudo -S tc qdisc show dev lo

A continuación ejecutamos intercom y evaluamos la calidad de la experiencia

Resultados

Finalmente borramos las reglas

In [ ]:
!echo {getpass()} | sudo -S tc qdisc delete dev lo root handle 1: tbf rate 4634kbit burst 32kbit limit 32kbit
!echo {getpass()} | sudo -S tc qdisc delete dev lo parent 1:1 handle 10: netem delay 43.75ms 6.88ms 0.375% distribution normal

Verificamos que se han eliminado las reglas

In [ ]:
!echo {getpass()} | sudo -S tc qdisc show dev lo

# Consideraciones finales

El bitrate de minimal intercom viene dado por:

$$bitrate = samplerate \cdot number\_channels \cdot bits\_per\_sample$$

Con los valores por defecto se tiene un bitrate aproximado de 1411 kbps. 

Si el bitrate de la conexión es menor que el bitrate de intercom $\rightarrow$ No se escuchará nada (o solamente ruido).

Si el bitrate es mayor (o igual) que el bitrate de intercom $\rightarrow$ Se escuchará, pero cuanto más próximo sea al valor del itrate de intercom, más frecuente será la pérdida de paquetes.
